In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

#### Пусть есть датасет людей, которые владеют неким предметом, который описывается параметрами длина, ширина, высота. 

In [70]:
# пусть количество записей равно 1000
n_samples = 1000
np.random.seed(33)

#добавляем параметры
#возраст владельца
age_owner = np.random.randint(20, 80, n_samples)
#рост владельца
height_owner_cm = np.random.randint(150, 190, n_samples)
#вес владельца
weight_owner_kg = np.random.randint(70, 110, n_samples)

#добавляем параметры предмета
length = np.random.randint(100, 500, n_samples)
width = np.random.randint(50, 90, n_samples)
height = np.random.randint(10, 45, n_samples)

#цена предмета описывается линейной зависимостью от параметров предмета
price = length * width * height * np.random.randint(100) + np.random.randint(400,800)

data = pd.DataFrame({'age_owner': age_owner, 'height_owner_cm':height_owner_cm, 'weight_owner_kg': weight_owner_kg, \
                     'length': length, 'width': width, 'height': height, 'price': price})
data.head(5)

,age_owner,height_owner_cm,weight_owner_kg,length,width,height,price
0,40,168,70,424,61,14,5431930
1,27,161,96,436,62,25,10137490
2,44,150,90,172,82,11,2327650
3,22,156,81,383,50,40,11490490
4,38,164,77,147,67,40,5909890


In [73]:
#узнать параметры моедли линейной регрессии
np.random.seed(33)
print(f"slope: {np.random.randint(100)}, intercept: {np.random.randint(400,800)}")

slope: 20, intercept: 791


In [59]:
# качестве метрики ошибки будем использовать средную абсолютную ошибку
from sklearn.metrics import mean_absolute_error

#Изначально предполагаем, что цена предмета зависит от всех признаков
#Х - исследуемые параметры
X = data[['age_owner', 'height_owner_cm', 'weight_owner_kg', 'length', 'width','height']]
#y - целевая переменная
y = data['price']

#задаем регрессию
reg = LinearRegression().fit(X, y)

print(f"Weights: {reg.coef_}") #коэффициенты линейной регрессии 
print(f"Bias: {reg.intercept_}") # свободный коэффициент линейной регрессии

pred_values = reg.predict(data[['age_owner', 'height_owner_cm', 'weight_owner_kg', \
                                'length', 'width','height']]) # предсказываем значения согласно построенной модели
print(f"Error {mean_absolute_error(pred_values, y)}")

Weights: [  4936.84224005  -1479.53895634   4455.80213606  28297.08401288
 127165.36004883 315243.46143326]
Bias: -17819905.060555644
Error 1051712.7246658402


#### Модель работает некорректно, ошибка огромная. Предположим что цена предмета не зависит от параметров владельца.

In [60]:
X = data[['length', 'width','height']]
y = data['price']

reg = LinearRegression().fit(X, y)

print(f"Weights: {reg.coef_}") 
print(f"Bias: {reg.intercept_}")

pred_values = reg.predict(data[['length', 'width','height']]) 
print(f"Error {mean_absolute_error(pred_values, y)}")

Weights: [ 28310.80291929 127325.58230876 314987.78448734]
Bias: -17436338.12852411
Error 1051023.9307760783


#### Модель всё также работает некорректно, ошибка огромная. Значит, что 1) либо цена предмета не зависит от каждого из параметров предмета отдельно (по типу модели k1x1 + k2x2 + k3x3 ...) 2) либо цена предмета всё же зависит от параметров владельца.
#### Предположим что цена модели зависит от произведения ее параметров

In [75]:
data['suggested_feature'] = data['length'] * data['width']*data['height']
data.head(5)

,age_owner,height_owner_cm,weight_owner_kg,length,width,height,price,suggested_feature
0,40,168,70,424,61,14,5431930,362096
1,27,161,96,436,62,25,10137490,675800
2,44,150,90,172,82,11,2327650,155144
3,22,156,81,383,50,40,11490490,766000
4,38,164,77,147,67,40,5909890,393960


In [63]:
X = data[['suggested_feature']]
y = data['price']

reg = LinearRegression().fit(X, y)

print(f"Weights: {reg.coef_}") 
print(f"Bias: {reg.intercept_}")

pred_values = reg.predict(data[['suggested_feature']]) 
print(f"Error {mean_absolute_error(pred_values, y)}")

Weights: [15.]
Bias: 489.9999999962747
Error 1.819804310798645e-09


#### Теперь модель работает корретно, ошибка минимальна. Сравним полученные значения в том случае, если для предсказания будем использовать дополнительные параметры.

In [78]:
X = data[['suggested_feature', 'length', 'width','height','age_owner','height_owner_cm']]
y = data['price']

reg = LinearRegression().fit(X, y)

print(f"Weights: {reg.coef_}") 
print(f"Bias: {reg.intercept_}")

pred_values = reg.predict(data[['suggested_feature', 'length', 'width','height','age_owner','height_owner_cm']]) 
print(f"Error {mean_absolute_error(pred_values, y)}")

Weights: [ 1.50000000e+01 -7.83702312e-13 -1.84951075e-12 -3.71294733e-12
  6.63794759e-13  6.57576263e-13]
Bias: 489.9999999925494
Error 3.28640453517437e-09


#### Даже при появлении лишних признаков модель работает корректно